## Project Overview

MLB Team Construction & Win Efficiency (2000–2016)

Modern MLB front offices operate under the constraint that spending more does not guarantee winning more. This project investigates how efficiently teams convert payroll into on-field success, using Wins Above Replacement (WAR) as a bridge between financial investment and team performance.

Using historical MLB data from 2000–2016, this analysis integrates:

Team wins and payroll from the Lahman Baseball Database (via SQLite)

Player-level WAR data aggregated to the team-season level using the JEFFBAGWELL historical WAR dataset

The core objective is to answer:

Which MLB teams were most efficient at converting payroll into WAR and wins—and how did team construction strategies differ across franchises and eras?

By combining relational SQL queries, Python data processing, and statistical modeling, the project evaluates:

The relationship between payroll, WAR, and wins

Differences in efficiency across teams and seasons

Whether WAR mediates the relationship between payroll and winning

Which organizations consistently outperformed their financial constraints

This project emphasizes reproducible data pipelines, clean feature engineering, and interpretable analytics, mirroring the workflows used in professional sports analytics and data science roles.

## Create the SQLite DB

In [18]:
import pandas as pd
import sqlite3
from pathlib import Path

# Paths
db_path = Path("../data/db/lahman.sqlite")
csv_path = Path("../data/raw")

# Create DB connection
conn = sqlite3.connect(db_path)

# Load CSVs
teams = pd.read_csv(csv_path / "Teams.csv")
salaries = pd.read_csv(csv_path / "Salaries.csv")

# Write to SQlite
teams.to_sql("Teams", conn, if_exists="replace", index=False)
salaries.to_sql("Salaries", conn, if_exists="replace", index=False)

# Verify
pd.read_sql("SELECT COUNT(*) AS rows From Teams", conn)



,rows
0,3614


In [19]:
pd.read_sql("SELECT COUNT(*) AS rows FROM Salaries", conn)

,rows
0,26428


## Build payroll per team-year

In [20]:
payroll_q = """ 
SELECT
    yearID,
    teamID,
    SUM(salary) AS payroll
FROM Salaries
WHERE yearID BETWEEN 2000 and 2016
GROUP BY yearID, teamID
"""
team_payroll = pd.read_sql(payroll_q, conn)
team_payroll.head()

,yearID,teamID,payroll
0,2000,ANA,51464167
1,2000,ARI,81027833
2,2000,ATL,84537836
3,2000,BAL,81447435
4,2000,BOS,77940333


In [21]:
team_payroll.sort_values("payroll", ascending=False).head(10)

,yearID,teamID,payroll
407,2013,NYA,231978886
403,2013,LAN,223362196
497,2016,NYA,222997792
493,2016,LAN,221288380
433,2014,LAN,217014600
463,2015,LAN,215792000
467,2015,NYA,212751957
167,2005,NYA,208306817
257,2008,NYA,207896789
317,2010,NYA,206333389


## Join payroll with wins

In [22]:
team_financials_q = """ 
SELECT
    t.yearID,
    t.teamID,
    t.name,
    t.W,
    p.payroll
FROM Teams t
JOIN (
    SELECT yearID, teamID, SUM(salary) AS payroll
    FROM Salaries
    WHERE yearID BETWEEN 2000 AND 2016
    GROUP BY yearID, teamID
)p
ON t.yearID = p.yearID AND t.teamID = p.teamID
WHERE t.yearID BETWEEN 2000 AND 2016
"""
team_financials = pd.read_sql(team_financials_q, conn)
team_financials.head()

,yearID,teamID,name,W,payroll
0,2000,ANA,Anaheim Angels,82,51464167
1,2000,ARI,Arizona Diamondbacks,85,81027833
2,2000,ATL,Atlanta Braves,95,84537836
3,2000,BAL,Baltimore Orioles,74,81447435
4,2000,BOS,Boston Red Sox,85,77940333


In [23]:
team_financials.shape


(510, 5)

In [24]:
from pathlib import Path
Path("../data/processed").mkdir(parents=True, exist_ok=True)

team_financials.to_csv("../data/processed/team_wins_payroll_2000_2024.csv", index=False)


## Team WAR

In [30]:

war_url = "https://raw.githubusercontent.com/Neil-Paine-1/MLB-WAR-data-historical/master/jeffbagwell_war_historical.csv"
war = pd.read_csv(
    war_url,
    encoding="latin1"
)

war.shape, war.columns[:15]

((112650, 78),
 Index(['player_name', 'age', 'key_bbref', 'year_ID', 'team_ID', 'stint_ID',
        'lg_ID', 'is_P', 'franch_ID', 'sched', 'g_bat', 'pa', 'bat162',
        'bsr162', 'fld162'],
       dtype='object'))

In [31]:
war.head()

,player_name,age,key_bbref,year_ID,team_ID,stint_ID,lg_ID,is_P,franch_ID,sched,...,gms_RF,gms_OF,gms_DH,gms_PH,gms_PR,prev_tm,arrived,departed,next_tm,salary
0,David Aardsma,22.0,aardsda01,2004,SFG,1,NL,Y,SFG,162,...,0.0,0.0,0.0,0.0,0.0,---,2004,2004,CHC,300000.0
1,David Aardsma,24.0,aardsda01,2006,CHC,1,NL,Y,CHC,162,...,0.0,0.0,0.0,0.0,0.0,SFG,2006,2006,CHW,NaN
2,David Aardsma,25.0,aardsda01,2007,CHW,1,AL,Y,CHW,162,...,0.0,0.0,0.0,0.0,0.0,CHC,2007,2007,BOS,387500.0
3,David Aardsma,26.0,aardsda01,2008,BOS,1,AL,Y,BOS,162,...,0.0,0.0,0.0,0.0,0.0,CHW,2008,2008,SEA,403250.0
4,David Aardsma,27.0,aardsda01,2009,SEA,1,AL,Y,SEA,162,...,0.0,0.0,0.0,0.0,0.0,BOS,2009,2010,NYY,419000.0


In [32]:
print(war.columns)


Index(['player_name', 'age', 'key_bbref', 'year_ID', 'team_ID', 'stint_ID',
       'lg_ID', 'is_P', 'franch_ID', 'sched', 'g_bat', 'pa', 'bat162',
       'bsr162', 'fld162', 'pos162', 'def162', 'rep162', 'bwar162', 'BB_plus',
       'K_plus', 'AVG_plus', 'OBP_plus', 'SLG_plus', 'wRC_plus', 'ISO_plus',
       'BABIP_plus', 'LD_plus', 'GB_plus', 'FB_plus', 'Pull_plus', 'Cent_plus',
       'Oppo_plus', 'g_pitch', 'starts', 'innings', 'relief_pct', 'avg_LI',
       'br_pwar162', 'fg_pwar162', 'ra9_pwar162', 'pwar162', 'K9_plus',
       'BB9_plus', 'KBB_plus', 'HR9_plus', 'Kpct_plus', 'BBpct_plus',
       'oppAVG_plus', 'WHIP_plus', 'oppBABIP_plus', 'LOB_plus', 'ERA_minus',
       'FIP_minus', 'xFIP_minus', 'oppLD_plus', 'oppGB_plus', 'oppFB_plus',
       'pct_PT', 'WAR162', 'gms_P', 'gms_C', 'gms_1B', 'gms_2B', 'gms_3B',
       'gms_SS', 'gms_LF', 'gms_CF', 'gms_RF', 'gms_OF', 'gms_DH', 'gms_PH',
       'gms_PR', 'prev_tm', 'arrived', 'departed', 'next_tm', 'salary'],
      dtype='object')

In [33]:
war_00_16 = war[(war["year_ID"] >= 2000) & (war["year_ID"] <= 2016)].copy()

## Aggregate player WAR to team-season WAR

In [34]:
team_war = (
    war_00_16
    .groupby(["year_ID", "team_ID"], as_index=False)[["bwar162", "pwar162", "WAR162"]]
    .sum()
    .rename(columns={
        "year_ID": "yearID",
        "team_ID": "teamID",
        "bwar162": "Batting_WAR",
        "pwar162": "Pitching_WAR",
        "WAR162": "Total_WAR"
    })
)

team_war.head(), team_war.shape

(   yearID teamID  Batting_WAR  Pitching_WAR  Total_WAR
 0    2000    ANA       30.250      8.410000  38.660000
 1    2000    ARI       13.965     20.973333  34.938333
 2    2000    ATL       24.530     21.163333  45.693333
 3    2000    BAL       17.065      8.560000  25.625000
 4    2000    BOS       15.940     24.290000  40.230000,
 (510, 5))

In [35]:
team_war.sort_values("Total_WAR", ascending=False).head(10)

,yearID,teamID,Batting_WAR,Pitching_WAR,Total_WAR
54,2001,SEA,47.425000,21.603333,69.028333
484,2016,CHC,34.695000,24.343333,59.038333
393,2013,BOS,37.765000,17.820000,55.585000
213,2007,BOS,29.405000,25.963333,55.368333
79,2002,NYY,28.339938,26.624348,54.964286
50,2001,OAK,28.745000,26.203333,54.948333
357,2011,TEX,33.760000,20.896667,54.656667
243,2008,BOS,31.900000,22.450000,54.350000
288,2009,NYY,33.730000,20.540000,54.270000
109,2003,NYY,28.658098,25.542331,54.200429


## Make team IDs consistent

In [36]:
lahman_ids = set(team_financials["teamID"].unique())
war_ids = set(team_war["teamID"].unique())

sorted(list(war_ids - lahman_ids))[:30], len(war_ids - lahman_ids)


(['CHC',
  'CHW',
  'FLA',
  'KCR',
  'LAD',
  'NYM',
  'NYY',
  'SDP',
  'SFG',
  'STL',
  'TBD',
  'TBR',
  'WSN'],
 13)

In [37]:
id_map = {
    # Modern -> Lahman IDs (what payroll table uses)
    "NYY": "NYA",
    "NYM": "NYN",
    "LAD": "LAN",
    "SFG": "SFN",
    "SDP": "SDN",
    "CHW": "CHA",
    "CHC": "CHN",
    "STL": "SLN",
    "WSN": "WAS",

    # Franchise changes / edge cases
    "FLA": "FLO",   # Marlins (Lahman often uses FLO in older ID schemes)
    "KCR": "KCA",   # Royals (older Lahman ID scheme)

    # Rays: Lahman might use either TBA or TBD so we’ll map both safely
    "TBR": "TBA",
    "TBD": "TBA",
}


In [39]:
team_war["teamID"] = team_war["teamID"].replace(id_map)  # overwrite
# ensure no leftover column
team_war = team_war.drop(columns=[c for c in ["teamID_lahman"] if c in team_war.columns])

In [40]:
final = team_financials.merge(
    team_war[["yearID", "teamID", "Batting_WAR", "Pitching_WAR", "Total_WAR"]],
    on=["yearID", "teamID"],
    how="left"
)

In [41]:
final["Total_WAR"].isna().mean()


np.float64(0.0)

## Efficiency Metrics

In [42]:
final = final.copy()

final["Dollars_per_Win"] = final["payroll"] / final["W"]
final["Dollars_per_WAR"] = final["payroll"] / final["Total_WAR"]

final["WAR_per_Million"] = final["Total_WAR"] / (final["payroll"] / 1_000_000)
final["Wins_per_100M"] = final["W"] / (final["payroll"] / 100_000_000)

final[["yearID","teamID","W","payroll","Total_WAR","Dollars_per_Win","Dollars_per_WAR"]].head()


,yearID,teamID,W,payroll,Total_WAR,Dollars_per_Win,Dollars_per_WAR
0,2000,ANA,82,51464167,38.660000,6.276118e+05,1.331199e+06
1,2000,ARI,85,81027833,34.938333,9.532686e+05,2.319167e+06
2,2000,ATL,95,84537836,45.693333,8.898720e+05,1.850113e+06
3,2000,BAL,74,81447435,25.625000,1.100641e+06,3.178436e+06
4,2000,BOS,85,77940333,40.230000,9.169451e+05,1.937368e+06


In [43]:
final.to_csv("../data/processed/team_wins_payroll_war_efficiency_2000_2016.csv", index=False)

In [44]:
final[["W","Total_WAR","payroll"]].corr()

,W,Total_WAR,payroll
W,1.000000,0.921338,0.342266
Total_WAR,0.921338,1.000000,0.334778
payroll,0.342266,0.334778,1.000000


In [45]:
final.sort_values("payroll", ascending=False).head(10)[["yearID","teamID","W","payroll","Total_WAR"]]

,yearID,teamID,W,payroll,Total_WAR
407,2013,NYA,85,231978886,28.658333
403,2013,LAN,92,223362196,47.583333
497,2016,NYA,84,222997792,32.671667
493,2016,LAN,91,221288380,40.135000
433,2014,LAN,94,217014600,49.790000
463,2015,LAN,92,215792000,46.806667
467,2015,NYA,87,212751957,39.558333
167,2005,NYA,95,208306817,38.128333
257,2008,NYA,89,207896789,38.971667
317,2010,NYA,95,206333389,50.328333
